In [0]:
checkpoints = spark.sql("""DESCRIBE EXTERNAL LOCATION `checkpoints`""").select('url').collect()[0][0]
bronze = spark.sql("""DESCRIBE EXTERNAL LOCATION `bronze`""").select('url').collect()[0][0]
silver = spark.sql("""DESCRIBE EXTERNAL LOCATION `silver`""").select('url').collect()[0][0]

In [0]:
print(checkpoints)

In [0]:
df_silverRoads = spark.readStream.table("`dev_catalog`.`bronze`.`raw_roads`")

In [0]:
def removeDuplicate(df):
    print('Remove duplication values: ',end='')
    df.dropDuplicates()
    print('Success!')
    return df

In [0]:
def handle_NULLs(df,columns):
    print('Handling NULL values on String cloumns with "unknown" ',end='')
    df_string = df.fillna('Unknown',subset=columns)
    print('Success!')

    print('Handling NULL values on String cloumns with "0" ',end='')
    df_clean = df_string.fillna(0,subset=columns)
    print('Success')

    return df_clean

In [0]:
df_noDup = removeDuplicate(df_silverRoads)
df_clean = handle_NULLs(df_noDup,df_noDup.schema.names)

### Creating road_category_name column

In [0]:
from pyspark.sql.functions import when,col
def road_Category(df):
    print('Creating Road Category name column',end='')
    df_roadCat=df.withColumn("Road_Category_Name",
                  when(col("Road_Category") == 'TA','Class A Trunk Road')
                  .when(col("Road_Category") == 'TM','Class A Trunk Motor')
                  .when(col("Road_Category") == 'PA','Class A Principal Road')
                  .when(col("Road_Category") == 'PM','Class A Principal Motor')
                  .when(col("Road_Category") == 'M','Class B road')
                  .otherwise('NA')
                  )
    print('Success')
    return df_roadCat

In [0]:
df_roadCat = road_Category(df_clean)

In [0]:
from pyspark.sql.functions import when,col
def road_Type(df):
    print('Creating Road Type name column',end='')
    df_roadType=df.withColumn("Road_Type",
                  when(col("Road_Category_Name").like('%Class A%'),'Major')
                  .when(col("Road_Category_Name").like('%Class B%'),'Minor')
                  .otherwise('NA')
                  )
    print('Success')
    return df_roadType

In [0]:
df_roadType = road_Type(df_roadCat)

In [0]:
df_roadType.writeStream \
    .format("delta") \
    .option("checkpointLocation",'abfss://checkpoints@datalakewarehouse52.dfs.core.windows.net/SilverRoadLoad/Checkpt') \
    .outputMode("append") \
    .trigger(availableNow=True) \
    .toTable("`dev_catalog`.`silver`.`silver_road`")